In [3]:
pip install torch_geometric networkx

  Obtaining dependency information for torch_geometric from https://files.pythonhosted.org/packages/1e/d3/4dffd7300500465e0b4a2ae917dcb2ce771de0b9a772670365799a27c024/torch_geometric-2.7.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/63.7 kB ? eta -:--:--
     ------------ ------------------------- 20.5/63.7 kB 330.3 kB/s eta 0:00:01
     ------------------ ------------------- 30.7/63.7 kB 262.6 kB/s eta 0:00:01
     ------------------------------------ - 61.4/63.7 kB 409.6 kB/s eta 0:00:01
     -------------------------------------- 63.7/63.7 kB 378.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.3 MB 1.9 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.3 MB 1.2 MB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.3 MB 1.4 MB/s eta 0:00:01
   ------- -------------------------------- 0.2/1.3 MB 1.5 MB/s eta 0:00:01
   -------- --------

In [4]:
import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),    # normal users community
         (3,4), (4,5), (5, 3)]    # suspicious cluster
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (e.g., train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

In [5]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Define GCN model for social network security (bot detection)
class SocialNetworkGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, 16)  # 1st layer: in_dim->16
        self.conv2 = GCNConv(16, num_classes)        # 2nd layer: 16->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

# Initialize model
model = SocialNetworkGCN(data.num_node_features, num_classes=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(f"Model initialized:")
print(f"  - Input features: {data.num_node_features}")
print(f"  - Number of classes: 2 (0=normal, 1=bot)")
print(f"  - Training nodes: {data.train_mask.sum().item()}")
print(f"  - Test nodes: {data.test_mask.sum().item()}")



Model initialized:
  - Input features: 6
  - Number of classes: 2 (0=normal, 1=bot)
  - Training nodes: 4
  - Test nodes: 2


In [6]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    # Print progress every 20 epochs
    if (epoch + 1) % 20 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

print("Training completed!")



Epoch 20/100, Loss: 0.2386
Epoch 40/100, Loss: 0.1010
Epoch 60/100, Loss: 0.0326
Epoch 80/100, Loss: 0.0131
Epoch 100/100, Loss: 0.0072
Training completed!


In [7]:
model.eval()
with torch.no_grad():
    pred = model(data).argmax(dim=1)
    acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
    
print(f'Test Accuracy: {acc:.2f}')
print(f'\nPredictions on test nodes:')
print(f'  Node 4: Predicted={pred[4].item()}, Actual={data.y[4].item()} ({"✓ Correct" if pred[4] == data.y[4] else "✗ Wrong"})')
print(f'  Node 5: Predicted={pred[5].item()}, Actual={data.y[5].item()} ({"✓ Correct" if pred[5] == data.y[5] else "✗ Wrong"})')



Test Accuracy: 1.00

Predictions on test nodes:
  Node 4: Predicted=1, Actual=1 (✓ Correct)
  Node 5: Predicted=1, Actual=1 (✓ Correct)
